<a href="https://colab.research.google.com/github/TsungmingLiu/AddressBook/blob/master/CIS5210_HW3_SEC3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3. Linear Disk Movement, Revisited [20 points]

Recall the Linear Disk Movement section from Homework 2. The starting configuration of this puzzle
is a row of ℓ cells, with disks located on cells 0 through n − 1. The goal is to move the disks to the
end of the row using a constrained set of actions. At each step, a disk can only be moved to an
adjacent empty cell, or to an empty cell two spaces away, provided another disk is located on the
intervening square.
In a variant of the problem, the disks were distinct rather than identical, and the goal state was
amended to stipulate that the final order of the disks should be the reverse of their initial order.
Implement an improved version of the solve_distinct_disks(length, n) function from Homework
2 that uses an A∗ search rather than an uninformed breadth-first search to find an optimal solution.
As before, the exact solution produced is not important so long as it is of minimal length. You should
devise a heuristic which is admissible but informative enough to yield significant improvements in
performance.
Hint: Recall that an admissable heuristic must not overestimate the distance to the goal. Review the
textbook’s example of relaxing the restrictions for tiles in the Eight Puzzle, and consider the rules of
disk movement of this puzzle.

In [ ]:
import heapq
import math

def get_next(state):
    length = len(state)
    for i in range(length):
        if state[i]:
            # Move to i+1 if empty
            if i + 1 < length and not state[i + 1]:
                next = list(state)
                next[i], next[i + 1] = state[i + 1], state[i]
                yield ((i, i + 1), tuple(next))

            # Jump over to i+2 if empty
            if i + 2 < length and state[i + 1] and not state[i + 2]:
                next = list(state)
                next[i], next[i + 2] = next[i + 2], next[i]
                yield ((i, i + 2), tuple(next))

            # Move to i-1 if empty
            if i - 1 >= 0 and not state[i - 1]:
                next = list(state)
                next[i], next[i - 1] = next[i - 1], next[i]
                yield ((i, i - 1), tuple(next))

            # Jump to i-2 if empty
            if i - 2 >= 0 and state[i - 1] and not state[i - 2]:
                next = list(state)
                next[i], next[i - 2] = next[i - 2], next[i]
                yield ((i, i - 2), tuple(next))

def dfs(start_state, goal_state):
    path = []
    queue = list([( [], start_state )])
    visited = {start_state}

    while queue:
        path, current_state = queue.pop(0)
        for move, next_state in get_next(current_state):
            if next_state not in visited:
                new_path = path + [move]
                if next_state == goal_state:
                    return new_path
                else:
                    visited.add(next_state)
                    queue.append((new_path, next_state))
    return []

def solve_distinct_disks_dfs(length, n):
    if n >= length or n == 0:
        return []

    start_state = tuple(range(1, n + 1)) + tuple([0] * (length - n))
    goal_state = tuple([0] * (length - n)) + tuple(reversed(range(1, n + 1)))

    return dfs(start_state, goal_state)

################################################################################################################
def heuristic(state, goal_positions):
    """
    The same admissible heuristic as before.
    """
    total_distance = 0
    for i, disk_id in enumerate(state):
        if disk_id != 0:
            goal_pos = goal_positions[disk_id]
            total_distance += abs(i - goal_pos)
    return total_distance / 2.0

def reconstruct_path(came_from, current_state):
    """
    Reconstructs the list of moves by tracing back from the goal state.
    """
    total_path = []
    while current_state in came_from:
        current_state, move = came_from[current_state]
        total_path.append(move)
    return total_path[::-1] # Reverse the path to get start -> goal order

def solve_distinct_disks(length, n):
    """
    Solves the distinct disks puzzle using a formal A* search with explicit
    open and closed sets.
    """
    if n >= length or n == 0:
        return []

    # --- Initialization ---
    start_state = tuple(range(1, n + 1)) + tuple([0] * (length - n))
    goal_state = tuple([0] * (length - n)) + tuple(reversed(range(1, n + 1)))
    goal_positions = {disk_id: length - disk_id for disk_id in range(1, n + 1)}

    # The set of discovered nodes that have not yet been evaluated (a priority queue)
    open_set = []
    heapq.heapify(open_set)

    # The set of nodes already evaluated
    closed_set = set()

    # For node n, came_from[n] is the node preceding it on the cheapest path
    came_from = {}

    # g_score[n] is the cost of the cheapest path from start to n
    g_score = {start_state: 0}

    # f_score[n] = g_score[n] + h(n)
    f_score = {start_state: heuristic(start_state, goal_positions)}

    # Push the start node onto the open set
    heapq.heappush(open_set, (f_score[start_state], start_state))

    # --- Search Loop ---
    while open_set:
        # Get the node in open_set having the lowest f_score
        current_f, current_state = heapq.heappop(open_set)

        # If we've reached the goal, reconstruct and return the path
        if current_state == goal_state:
            return reconstruct_path(came_from, current_state)

        # Move the current node from open to closed
        closed_set.add(current_state)

        # Explore neighbors
        for move, neighbor_state in get_next(current_state):
            # Ignore neighbors that have already been evaluated
            if neighbor_state in closed_set:
                continue

            # The distance from start to a neighbor is the distance to current + 1
            temp_g_score = g_score.get(current_state, math.inf) + 1

            # Discover a new node or find a better path
            if temp_g_score < g_score.get(neighbor_state, math.inf):
                came_from[neighbor_state] = (current_state, move)
                g_score[neighbor_state] = temp_g_score
                h_score = heuristic(neighbor_state, goal_positions)
                f_score[neighbor_state] = temp_g_score + h_score

                # Add the neighbor to the open set for evaluation
                heapq.heappush(open_set, (f_score[neighbor_state], neighbor_state))

    # If the open set is empty and the goal was not reached, no solution exists
    return []


print(solve_distinct_disks(4, 2))
# [(0, 2), (2, 3), (1, 2)]
print(solve_distinct_disks(5, 2))
# [(0, 2), (1, 3), (2, 4)]
print(solve_distinct_disks(4, 3))
# [(1, 3), (0, 1), (2, 0), (3, 2), (1, 3), (0, 1)]
print(solve_distinct_disks(5, 3))
# [(1, 3), (2, 1), (0, 2), (2, 4), (1, 2)]




[(0, 2), (2, 3), (1, 2)]
[(0, 2), (1, 3), (2, 4)]
[(1, 3), (0, 1), (2, 0), (3, 2), (1, 3), (0, 1)]
[(1, 3), (2, 1), (0, 2), (2, 4), (1, 2)]
